# AgentV1

> Fill in a module description here

In [1]:
#| default_exp AgentV1

In [2]:
#|export
from Text2SQL.SnowflakeCore import *
from lisette import *
import pandas as pd
import ast
import regex as re
import json

In [3]:
#|export
from IPython.display import Markdown, display
MD = lambda x: display(Markdown(x))

## System Prompt

In [4]:
#|export
def create_system_prompt(schema: ParentSchema) -> str:
    return f"""You are a SQL query generator for {schema.dialect}.

DATABASE: {schema.database}.{schema.Schema}
SCHEMA: {schema.model_dump_json(indent=2)}

CORE RULES:
1. Generate SELECT queries only - no modifications
2. ALWAYS quote identifiers: SELECT "column_name" FROM "table_name"
3. Use fully qualified names: {schema.database}.{schema.Schema}."TABLE_NAME"
4. No semicolons - single statement only
5. Use the execute_query tool to run queries
6. Format queries in triple quotes for readability
7. If user asks for all data do not apply `limit`

JSON COLUMNS:
- Many columns store JSON as VARCHAR (see samples)
- To extract values: GET(PARSE_JSON("column_name"), 'key')::VARCHAR
- Example: GET(PARSE_JSON("airport_name"), 'en')::VARCHAR

NULL VALUES:
- Sample data shows '\\N' but use IS NULL / IS NOT NULL in WHERE clauses

JSON COLUMNS:
- Many columns store JSON as VARCHAR (city, airport_name, model)
- Extract English values: GET(PARSE_JSON("column_name"), 'en')::VARCHAR
- Example: GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name"

ERROR RECOVERY:
If query fails:
1. Check quoted identifiers (all table/column names)
2. Verify table names are fully qualified
3. Confirm JSON extraction syntax
4. Check for typos in column/table names
5. Retry with corrected query

COMMON PATTERNS:
- Aggregations: Use COUNT(*), SUM(), AVG() with GROUP BY
- Top N: Use ORDER BY ... LIMIT N
- Deduplication: Use DISTINCT or GROUP BY
- Date filtering: Use BETWEEN or comparison operators on date columns

PERFORMANCE:
- Always use LIMIT for exploratory queries
- Prefer specific columns over SELECT *
- Use indexes via foreign key relationships

Be precise, efficient, and always validate your SQL syntax before execution."""


## Experiment

In [5]:
agent = SnowflakeAgent()

M = SnowMetadata(
    agent,
    "AIRLINES", "AIRLINES", model_name = model_name)
assert M()

In [6]:
SYSTEM_PROMPT = create_system_prompt(M.metadata)
print(SYSTEM_PROMPT)

You are a SQL query generator for snowflake.

DATABASE: AIRLINES.AIRLINES
SCHEMA: {
  "dialect": "snowflake",
  "database": "AIRLINES",
  "Schema": "AIRLINES",
  "tables": [
    {
      "name": "AIRLINES.AIRLINES.AIRCRAFTS_DATA",
      "column_names": [
        {
          "name": "aircraft_code",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": "model",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": "range",
          "type": "NUMBER(38,0)",
          "null?": "Y"
        }
      ],
      "sample_rows": [
        {
          "aircraft_code": "773",
          "model": "{\"en\": \"Boeing 777-300\", \"ru\": \"Боинг 777-300\"}",
          "range": 11100
        }
      ],
      "row_count": 9
    },
    {
      "name": "AIRLINES.AIRLINES.AIRPORTS_DATA",
      "column_names": [
        {
          "name": "airport_code",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
   

In [7]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
res = chat("Which airport has the most departures?", max_steps=5)
res

The airport with the most departures is **Domodedovo International Airport** (airport code: DME) with **3,217** departures.

<details>

- id: `chatcmpl-xzr0kmzg7ingg0c8921v`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=44, prompt_tokens=3656, total_tokens=3700, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [8]:
chat.print_hist()

{'role': 'user', 'content': 'Which airport has the most departures?'}

Message(content='', role='assistant', tool_calls=[{'function': {'arguments': '{"query":"SELECT GET(PARSE_JSON(\\"airport_name\\"), \'en\')::VARCHAR AS \\"airport_name\\", COUNT(*) AS \\"departure_count\\"\\nFROM AIRLINES.AIRLINES.\\"FLIGHTS\\"\\nGROUP BY \\"departure_airport\\"\\nORDER BY \\"departure_count\\" DESC\\nLIMIT 1"}', 'name': 'execute_query'}, 'id': '688635713', 'type': 'function'}], function_call=None, reasoning_content='Need count of flights per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport. Order desc limit 1. Extract airport_name en. Need join to AIRPORTS_DATA. Query.', provider_specific_fields={'refusal': None, 'reasoning': 'Need count of flights per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport. Order desc limit 1. Extract airport_name en

In [9]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
res = chat("Which airport has the least departures?", max_steps=5)
res

The airport with the fewest departures is **Usinsk Airport** (code: *USK*), with only **18** scheduled departures in the dataset.

<details>

- id: `chatcmpl-l62ynxwfmnq4loga92fvfd`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=47, prompt_tokens=3683, total_tokens=3730, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [10]:
chat.print_hist()

{'role': 'user', 'content': 'Which airport has the least departures?'}

Message(content='', role='assistant', tool_calls=[{'function': {'arguments': '{"query":"SELECT GET(PARSE_JSON(\\"airport_name\\"), \'en\')::VARCHAR AS \\"airport_name\\", COUNT(*) AS \\"departure_count\\"\\nFROM AIRLINES.AIRLINES.\\"FLIGHTS\\"\\nGROUP BY \\"departure_airport\\"\\nORDER BY \\"departure_count\\" ASC\\nLIMIT 1"}', 'name': 'execute_query'}, 'id': '381813769', 'type': 'function'}], function_call=None, reasoning_content='Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also extract airport_name en. Need join to AIRPORTS_DATA. Query.', provider_specific_fields={'refusal': None, 'reasoning': 'Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also ext

## For formtting user input


In [11]:
chat.hist[0]

{'role': 'user', 'content': 'Which airport has the least departures?'}

In [12]:
#|export
def md_user(msg):
    """Format user message with quote styling"""
    content = msg.get('content', '') if isinstance(msg, dict) else msg.content
    
    return f"""### 👤 **User**

> {content}
"""

In [13]:
MD(md_user(chat.hist[0]))

### 👤 **User**

> Which airport has the least departures?


## For formtting  tool execute query response

Implement rendering of chat  history for better visualization and analysis.

In [14]:
content = chat.hist[2]['content']

# Extract the key-value pairs
# This regex finds pattern: key=value
pattern = r"(\w+)=(.+?)(?=\s+\w+=|\s*$)"
matches = re.findall(pattern, content)
matches

[('query',
  '\'SELECT GET(PARSE_JSON("airport_name"), \\\'en\\\')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"\\nFROM AIRLINES.AIRLINES."FLIGHTS"\\nGROUP BY "departure_airport"\\nORDER BY "departure_count" ASC\\nLIMIT 1\''),
 ('success', 'False'),
 ('data', 'None'),
 ('error',
  '\'000904 (42000): SQL compilation error: error line 1 at position 22\\ninvalid identifier \\\'"airport_name"\\\'\''),
 ('row_count', '0'),
 ('execution_time', '0.0')]

In [15]:
ast.literal_eval(matches[2][1].strip())

In [16]:
chat.hist[2]

{'tool_call_id': '381813769',
 'role': 'tool',
 'name': 'execute_query',
 'content': 'query=\'SELECT GET(PARSE_JSON("airport_name"), \\\'en\\\')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"\\nFROM AIRLINES.AIRLINES."FLIGHTS"\\nGROUP BY "departure_airport"\\nORDER BY "departure_count" ASC\\nLIMIT 1\' success=False data=None error=\'000904 (42000): SQL compilation error: error line 1 at position 22\\ninvalid identifier \\\'"airport_name"\\\'\' row_count=0 execution_time=0.0'}

We want the details of content. It is a string object.

In [17]:
#|export
def str2key_value(msg:str):
    """
    Extracts key and value from a message string.

    Args:
        msg (str): The message string to extract tools from.
        Ex: `'query=\'SELECT "departure_airport", COUNT(*) AS "departures" FROM AIRLINES.AIRLINES."FLIGHTS" WHERE "scheduled_departure" IS NOT NULL GROUP BY "departure_airport" ORDER BY "departures" ASC LIMIT 1\' success=True data=[{\'departure_airport\': \'USK\', \'departures\': 18}] error=None row_count=1 execution_time=0.3334343433380127'`

    Returns:
        dict: A dictionary of tools extracted from the message string.
        Ex:
        {'query': 'SELECT "departure_airport", COUNT(*) AS "departures" FROM AIRLINES.AIRLINES."FLIGHTS" WHERE "scheduled_departure" IS NOT NULL GROUP BY "departure_airport" ORDER BY "departures" ASC LIMIT 1',
        'success': True,
        'data': [{'departure_airport': 'USK', 'departures': 18}],
        'error': None,
        'row_count': 1,
        'execution_time': 0.3334343433380127}

    """
    pattern = r"(\w+)=(.+?)(?=\s+\w+=|\s*$)"
    matches = re.findall(pattern, msg)
    parsed = {}
    for key, value in matches:
        try:
            # Try to evaluate as Python literal
            parsed[key] = ast.literal_eval(value.strip())
        except (ValueError, SyntaxError):
            # If it fails, keep as string
            parsed[key] = value.strip()
    return parsed

In [18]:
str2key_value('content')

{}

In [19]:
dic = str2key_value(chat.hist[2]['content'])
dic

{'query': 'SELECT GET(PARSE_JSON("airport_name"), \'en\')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"\nFROM AIRLINES.AIRLINES."FLIGHTS"\nGROUP BY "departure_airport"\nORDER BY "departure_count" ASC\nLIMIT 1',
 'success': False,
 'data': None,
 'error': '000904 (42000): SQL compilation error: error line 1 at position 22\ninvalid identifier \'"airport_name"\'',
 'row_count': 0,
 'execution_time': 0.0}

In [20]:
#|export
def md_tool(dic):
    """Compact tool result formatting"""
    content = str2key_value(dic.get('content', ''))
    
    success = content.get('success', False)
    data = content.get('data', [])
    row_count = content.get('row_count', 0)
    exec_time = content.get('execution_time', 0)
    
    base_str = f"""### 🛠️ **Tool**
__Executed: {dic.get('name', '')}__

"""
    
    if not success:
        return base_str + f"❌ **Error:** {content.get('error')}"
    
    # Format first few rows
    if data: 
        if isinstance(data, str):
            # rather than conerting str to table
            # better to refire the query ti db 
            query = content.get('query', '')
            r = agent.execute_query(query)
            df = pd.json_normalize(r.data)
        else:
            df = pd.json_normalize(data[:5])
        
        table = df.to_markdown(index=False)
    else:
        table = "_No data_"
    
    return base_str + f"""✅ **Result** ({row_count} rows, {exec_time:.2f}s)

{table}
"""

In [21]:
MD(md_tool(chat.hist[2]))

### 🛠️ **Tool**
__Executed: execute_query__

❌ **Error:** 000904 (42000): SQL compilation error: error line 1 at position 22
invalid identifier '"airport_name"'

## For formtting  assistant response

In [22]:
#|export
from litellm.types.utils import Message
def md_assistant(msg:Message):
    """Format message as markdown - handles both tool calls and regular responses"""
    
    # Check if this is a tool call message
    if msg.tool_calls and len(msg.tool_calls) > 0:
        # Format tool call
        try:
            func = msg.tool_calls[0]['function']
            args = json.loads(func['arguments'])
            query = args.pop('query', '')
            
            query_section = f"""<details>
<summary><b>📝 Query</b></summary>

```sql
{query}

```
</details>
"""  if query else ''
            #query_section = f">**Query:**\n```sql\n{query}\n```\n" if query else ''
            
            args_table = (
                "\n| Parameter | Value |\n|-----------|-------|\n" + 
                "\n".join(f"| **{k}** | `{v}` |" for k, v in args.items())
            ) if args else ""
            
            reasoning = msg.reasoning_content or "No reasoning provided"
            
            return f"""### 🧠 Assistant \n\n
_Reasoning: {reasoning}_

**Call: {func['name']}**

{query_section}{args_table}

"""
        except (IndexError, KeyError, json.JSONDecodeError) as e:
            return f"❌ Error formatting tool call: {e}"
    
    else:
        # Regular assistant message (no tool calls)
        reasoning = msg.reasoning_content or ""
        reasoning_section = f"\n\n💭 _Reasoning: {reasoning}_" if reasoning else ""
        
        return f"""🤖 **Assistant**

{msg.content}{reasoning_section}
"""

In [23]:
m = md_assistant(chat.hist[1])
display(Markdown(m))

### 🧠 Assistant 


_Reasoning: Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also extract airport_name en. Need join to AIRPORTS_DATA. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"
FROM AIRLINES.AIRLINES."FLIGHTS"
GROUP BY "departure_airport"
ORDER BY "departure_count" ASC
LIMIT 1

```
</details>




In [24]:
MD(md_assistant(chat.hist[-1]))

🤖 **Assistant**

The airport with the fewest departures is **Usinsk Airport** (code: *USK*), with only **18** scheduled departures in the dataset.

💭 _Reasoning: Need to output result._


## Putting it all together

In [25]:
r = []
for h in chat.hist:
    if isinstance(h, dict) :
        ## it will be for user and tools
        if h['role'] == 'user':
            r.append(md_user(h))
        elif h['role'] == 'tool':
            r.append(md_tool(h))
    elif isinstance(h, Message):
        ## assitant turn
        r.append(md_assistant(h))

In [26]:
MD('\n ---\n'.join(r))

### 👤 **User**

> Which airport has the least departures?

 ---
### 🧠 Assistant 


_Reasoning: Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also extract airport_name en. Need join to AIRPORTS_DATA. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"
FROM AIRLINES.AIRLINES."FLIGHTS"
GROUP BY "departure_airport"
ORDER BY "departure_count" ASC
LIMIT 1

```
</details>



 ---
### 🛠️ **Tool**
__Executed: execute_query__

❌ **Error:** 000904 (42000): SQL compilation error: error line 1 at position 22
invalid identifier '"airport_name"'
 ---
### 🧠 Assistant 


_Reasoning: Need airport_name from AIRPORTS_DATA join. Use departure_airport code._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"
FROM AIRLINES.AIRLINES."FLIGHTS"
JOIN AIRLINES.AIRLINES."AIRPORTS_DATA" ON "FLIGHTS"."departure_airport" = "AIRPORTS_DATA"."airport_code"
GROUP BY "departure_airport", GET(PARSE_JSON("airport_name"), 'en')::VARCHAR
ORDER BY "departure_count" ASC
LIMIT 1

```
</details>



 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.62s)

| airport_name   |   departure_count |
|:---------------|------------------:|
| Usinsk Airport |                18 |

 ---
🤖 **Assistant**

The airport with the fewest departures is **Usinsk Airport** (code: *USK*), with only **18** scheduled departures in the dataset.

💭 _Reasoning: Need to output result._


In [27]:
def vis(chat):
        r = []
        for h in chat.hist:
            if isinstance(h, dict) :
                ## it will be for user and tools
                if h['role'] == 'user':
                    r.append(md_user(h))
                elif h['role'] == 'tool':
                    r.append(md_tool(h))
            elif isinstance(h, Message):
                ## assitant turn
                r.append(md_assistant(h))
        return MD('\n ---\n'.join(r))

In [28]:
vis(chat)

### 👤 **User**

> Which airport has the least departures?

 ---
### 🧠 Assistant 


_Reasoning: Need count of departures per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport and order asc limit 1. Also extract airport_name en. Need join to AIRPORTS_DATA. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"
FROM AIRLINES.AIRLINES."FLIGHTS"
GROUP BY "departure_airport"
ORDER BY "departure_count" ASC
LIMIT 1

```
</details>



 ---
### 🛠️ **Tool**
__Executed: execute_query__

❌ **Error:** 000904 (42000): SQL compilation error: error line 1 at position 22
invalid identifier '"airport_name"'
 ---
### 🧠 Assistant 


_Reasoning: Need airport_name from AIRPORTS_DATA join. Use departure_airport code._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"
FROM AIRLINES.AIRLINES."FLIGHTS"
JOIN AIRLINES.AIRLINES."AIRPORTS_DATA" ON "FLIGHTS"."departure_airport" = "AIRPORTS_DATA"."airport_code"
GROUP BY "departure_airport", GET(PARSE_JSON("airport_name"), 'en')::VARCHAR
ORDER BY "departure_count" ASC
LIMIT 1

```
</details>



 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.62s)

| airport_name   |   departure_count |
|:---------------|------------------:|
| Usinsk Airport |                18 |

 ---
🤖 **Assistant**

The airport with the fewest departures is **Usinsk Airport** (code: *USK*), with only **18** scheduled departures in the dataset.

💭 _Reasoning: Need to output result._


## Other Expriments

### 1.

In [29]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
res = chat("How many flights are there?", max_steps=5)
res

The total number of flights is **33,121**.

<details>

- id: `chatcmpl-t8k0uyv1a3gwjhthhn1v9`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=16, prompt_tokens=3308, total_tokens=3324, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [30]:
vis(chat)

### 👤 **User**

> How many flights are there?

 ---
### 🧠 Assistant 


_Reasoning: Need count from FLIGHTS. Use fully qualified._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES."FLIGHTS"

```
</details>



 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.35s)

|   flight_count |
|---------------:|
|          33121 |

 ---
🤖 **Assistant**

The total number of flights is **33,121**.


### 2

In [31]:
chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
res = chat("Which airport has the most departures?", max_steps=5)
res

The airport with the most departures is **Domodedovo International Airport** (airport code: DME) with **3,217** departure flights.

<details>

- id: `chatcmpl-xlh2sxyjwnnwgtxgijmuip`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=45, prompt_tokens=3656, total_tokens=3701, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [32]:
vis(chat)

### 👤 **User**

> Which airport has the most departures?

 ---
### 🧠 Assistant 


_Reasoning: Need count of flights per departure_airport. Use FLIGHTS table. Count distinct flight_id? Probably each flight counts once. So group by departure_airport. Order desc limit 1. Extract airport_name en. Need join to AIRPORTS_DATA. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"
FROM AIRLINES.AIRLINES."FLIGHTS"
GROUP BY "departure_airport"
ORDER BY "departure_count" DESC
LIMIT 1

```
</details>



 ---
### 🛠️ **Tool**
__Executed: execute_query__

❌ **Error:** 000904 (42000): SQL compilation error: error line 1 at position 22
invalid identifier '"airport_name"'
 ---
### 🧠 Assistant 


_Reasoning: Need airport_name from AIRPORTS_DATA join. Use departure_airport code. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("airport_name"), 'en')::VARCHAR AS "airport_name", COUNT(*) AS "departure_count"
FROM AIRLINES.AIRLINES."FLIGHTS"
JOIN AIRLINES.AIRLINES."AIRPORTS_DATA" ON "FLIGHTS"."departure_airport" = "AIRPORTS_DATA"."airport_code"
GROUP BY "airport_name"
ORDER BY "departure_count" DESC
LIMIT 1

```
</details>



 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (1 rows, 0.37s)

| airport_name                     |   departure_count |
|:---------------------------------|------------------:|
| Domodedovo International Airport |              3217 |

 ---
🤖 **Assistant**

The airport with the most departures is **Domodedovo International Airport** (airport code: DME) with **3,217** departure flights.

💭 _Reasoning: Need to output result._


### 3

In [33]:
def ask(q):
    chat = Chat(model_name, sp=SYSTEM_PROMPT, tools=[agent.execute_query])
    res = chat(q, max_steps=15)
    return chat, res

In [34]:
q = "What are the top 5 busiest routes by number of flights?"

chat, res = ask(q)
res

Here are the top 5 busiest routes (by number of flights) in the dataset:

| # | Departure Airport | Arrival Airport | Flight Count |
|---|--------------------|-----------------|--------------|
| 1 | SVO                | LED             | 305          |
| 2 | LED                | SVO             | 305          |
| 3 | DME                | LED             | 244          |
| 4 | LED                | DME             | 244          |
| 5 | BZK                | SVO             | 183          |

These counts are derived from the `FLIGHTS` table, grouping by both departure and arrival airports.

<details>

- id: `chatcmpl-hwr1y282ox5ocwhrpl2op`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=150, prompt_tokens=3498, total_tokens=3648, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [35]:
vis(chat)

### 👤 **User**

> What are the top 5 busiest routes by number of flights?

 ---
### 🧠 Assistant 


_Reasoning: Need query joining FLIGHTS and maybe count distinct flight_id per route (departure_airport, arrival_airport). Order desc limit 5. Use fully qualified names. Return columns: departure_airport, arrival_airport, COUNT(*) AS "flight_count"._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT "departure_airport", "arrival_airport", COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES."FLIGHTS" GROUP BY "departure_airport", "arrival_airport" ORDER BY "flight_count" DESC LIMIT 5

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (5 rows, 0.37s)

| departure_airport   | arrival_airport   |   flight_count |
|:--------------------|:------------------|---------------:|
| SVO                 | LED               |            305 |
| LED                 | SVO               |            305 |
| DME                 | LED               |            244 |
| LED                 | DME               |            244 |
| BZK                 | SVO               |            183 |

 ---
🤖 **Assistant**

Here are the top 5 busiest routes (by number of flights) in the dataset:

| # | Departure Airport | Arrival Airport | Flight Count |
|---|--------------------|-----------------|--------------|
| 1 | SVO                | LED             | 305          |
| 2 | LED                | SVO             | 305          |
| 3 | DME                | LED             | 244          |
| 4 | LED                | DME             | 244          |
| 5 | BZK                | SVO             | 183          |

These counts are derived from the `FLIGHTS` table, grouping by both departure and arrival airports.

💭 _Reasoning: Return results._


### 4

In [36]:
q = "Which aircraft models have the most seats available?"

chat, res = ask(q)
res

Here are the aircraft models with the highest number of seats available, ordered from most to least:

| Model | Seat Count |
|-------|------------|
| Boeing 777‑300 | 402 |
| Boeing 767‑300 | 222 |
| Airbus A321‑200 | 170 |
| Airbus A320‑200 | 140 |
| Boeing 737‑300 | 130 |
| Airbus A319‑100 | 116 |
| Sukhoi Superjet‑100 | 97 |
| Bombardier CRJ‑200 | 50 |
| Cessna 208 Caravan | 12 |

These counts were derived by joining the `SEATS` table with `AIRCRAFTS_DATA`, extracting the English model name from the JSON column, and aggregating the seat rows per model.

<details>

- id: `chatcmpl-42l9vplnqvv1cvtan0g8hy`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=181, prompt_tokens=3818, total_tokens=3999, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [37]:
vis(chat)

### 👤 **User**

> Which aircraft models have the most seats available?

 ---
### 🧠 Assistant 


_Reasoning: Need join SEATS with AIRCRAFTS_DATA to count seats per model. Extract model en. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("model"),'en')::VARCHAR AS "model", COUNT(*) AS "seat_count"
FROM AIRLINES.AIRLINES."SEATS"
GROUP BY GET(PARSE_JSON("model"),'en')::VARCHAR
ORDER BY "seat_count" DESC

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

❌ **Error:** 000904 (42000): SQL compilation error: error line 1 at position 22
invalid identifier '"model"'
 ---
### 🧠 Assistant 


_Reasoning: Need join SEATS with AIRCRAFTS_DATA to get model. Use aircraft_code._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON(a."model"),'en')::VARCHAR AS "model", COUNT(*) AS "seat_count"
FROM AIRLINES.AIRLINES."SEATS" s
JOIN AIRLINES.AIRLINES."AIRCRAFTS_DATA" a ON s."aircraft_code" = a."aircraft_code"
GROUP BY GET(PARSE_JSON(a."model"),'en')::VARCHAR
ORDER BY "seat_count" DESC

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (9 rows, 74.53s)

| model           |   seat_count |
|:----------------|-------------:|
| Boeing 777-300  |          402 |
| Boeing 767-300  |          222 |
| Airbus A321-200 |          170 |
| Airbus A320-200 |          140 |
| Boeing 737-300  |          130 |

 ---
🤖 **Assistant**

Here are the aircraft models with the highest number of seats available, ordered from most to least:

| Model | Seat Count |
|-------|------------|
| Boeing 777‑300 | 402 |
| Boeing 767‑300 | 222 |
| Airbus A321‑200 | 170 |
| Airbus A320‑200 | 140 |
| Boeing 737‑300 | 130 |
| Airbus A319‑100 | 116 |
| Sukhoi Superjet‑100 | 97 |
| Bombardier CRJ‑200 | 50 |
| Cessna 208 Caravan | 12 |

These counts were derived by joining the `SEATS` table with `AIRCRAFTS_DATA`, extracting the English model name from the JSON column, and aggregating the seat rows per model.

💭 _Reasoning: Need to present results._


### 5

In [38]:
q = "What is the average booking amount by all months?"
chat, res = ask(q)
res

```sql
SELECT EXTRACT(MONTH FROM TO_TIMESTAMP("book_date")) AS "month",
       AVG("total_amount") AS "avg_total_amount"
FROM AIRLINES.AIRLINES."BOOKINGS"
GROUP BY 1
ORDER BY 1;
```

Result (first 10 rows):

| month | avg_total_amount |
|-------|------------------|
| 6     | 76319.457757     |
| 7     | 79217.606637     |
| 8     | 78886.354229     |

(Only three months are present in the sample data.)

<details>

- id: `chatcmpl-4y1bpe1itrbrvqcyyirqkl`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=139, prompt_tokens=3656, total_tokens=3795, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [39]:
vis(chat)

### 👤 **User**

> What is the average booking amount by all months?

 ---
### 🧠 Assistant 


_Reasoning: Need avg total_amount per month. Use BOOKINGS.book_date cast to date and extract month. Query._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT GET(PARSE_JSON("book_date"), 'en')::VARCHAR AS "book_date", EXTRACT(MONTH FROM TO_TIMESTAMP("book_date")) AS "month", AVG("total_amount") AS "avg_total_amount"
FROM AIRLINES.AIRLINES."BOOKINGS"
GROUP BY 1,2
ORDER BY 2

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

❌ **Error:** 100069 (22P02): Error parsing JSON: garbage in the numeric literal: 2017-07-05 , pos 11
 ---
### 🧠 Assistant 


_Reasoning: Need to parse book_date as timestamp directly, not JSON. It's string. So use TO_TIMESTAMP(book_date). Also no need GET. Let's try._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT EXTRACT(MONTH FROM TO_TIMESTAMP("book_date")) AS "month", AVG("total_amount") AS "avg_total_amount"
FROM AIRLINES.AIRLINES."BOOKINGS"
GROUP BY 1
ORDER BY 1

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (3 rows, 0.38s)

|   month |   avg_total_amount |
|--------:|-------------------:|
|       6 |            76319.5 |
|       7 |            79217.6 |
|       8 |            78886.4 |

 ---
🤖 **Assistant**

```sql
SELECT EXTRACT(MONTH FROM TO_TIMESTAMP("book_date")) AS "month",
       AVG("total_amount") AS "avg_total_amount"
FROM AIRLINES.AIRLINES."BOOKINGS"
GROUP BY 1
ORDER BY 1;
```

Result (first 10 rows):

| month | avg_total_amount |
|-------|------------------|
| 6     | 76319.457757     |
| 7     | 79217.606637     |
| 8     | 78886.354229     |

(Only three months are present in the sample data.)

💭 _Reasoning: Need to output result._


In [40]:
q = "Business class revenue wrt most used route"
chat, res = ask(q)
res

**Business‑class revenue by most used route**

```sql
SELECT "departure_airport" AS "from",
       "arrival_airport"   AS "to",
       SUM("amount")        AS "business_revenue"
FROM AIRLINES.AIRLINES."TICKET_FLIGHTS"
JOIN AIRLINES.AIRLINES."FLIGHTS"
  ON "TICKET_FLIGHTS"."flight_id" = "FLIGHTS"."flight_id"
WHERE "fare_conditions" = 'Business'
GROUP BY "departure_airport", "arrival_airport"
ORDER BY "business_revenue" DESC
LIMIT 10;
```

**Result (top 10 routes)**  

| from | to | business_revenue |
|------|----|------------------|
| DME  | KHV | 237,082,500 |
| KHV  | DME | 233,577,000 |
| KHV  | LED | 159,945,800 |
| LED  | KHV | 156,408,000 |
| LED  | IKT | 151,771,800 |
| IKT  | LED | 150,974,400 |
| DME  | OVB | 99,184,500 |
| OVB  | DME | 97,594,200 |
| IKT  | KZN | 89,167,800 |
| KZN  | IKT | 88,956,000 |

These are the routes generating the highest business‑class revenue in the dataset.

<details>

- id: `chatcmpl-n1mm9u4dng9obipdreezkn`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=335, prompt_tokens=3712, total_tokens=4047, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [41]:
vis(chat)

### 👤 **User**

> Business class revenue wrt most used route

 ---
### 🧠 Assistant 


_Reasoning: Need query: sum amount per route (departure_airport-arrival_airport) for business fare_conditions. Use TICKET_FLIGHTS join FLIGHTS to get airports. Sum amount where fare_conditions='Business'. Group by route. Order desc limit maybe 10._

**Call: execute_query**

<details>
<summary><b>📝 Query</b></summary>

```sql
SELECT "departure_airport" AS "from", "arrival_airport" AS "to", SUM("amount") AS "business_revenue" FROM AIRLINES.AIRLINES."TICKET_FLIGHTS" JOIN AIRLINES.AIRLINES."FLIGHTS" ON "TICKET_FLIGHTS"."flight_id" = "FLIGHTS"."flight_id" WHERE "fare_conditions" = 'Business' GROUP BY "departure_airport", "arrival_airport" ORDER BY "business_revenue" DESC LIMIT 10

```
</details>

| Parameter | Value |
|-----------|-------|
| **max_rows** | `10` |


 ---
### 🛠️ **Tool**
__Executed: execute_query__

✅ **Result** (10 rows, 1.10s)

| from   | to   |   business_revenue |
|:-------|:-----|-------------------:|
| DME    | KHV  |          237082500 |
| KHV    | DME  |          233577000 |
| KHV    | LED  |          159945800 |
| LED    | KHV  |          156408000 |
| LED    | IKT  |          151771800 |

 ---
🤖 **Assistant**

**Business‑class revenue by most used route**

```sql
SELECT "departure_airport" AS "from",
       "arrival_airport"   AS "to",
       SUM("amount")        AS "business_revenue"
FROM AIRLINES.AIRLINES."TICKET_FLIGHTS"
JOIN AIRLINES.AIRLINES."FLIGHTS"
  ON "TICKET_FLIGHTS"."flight_id" = "FLIGHTS"."flight_id"
WHERE "fare_conditions" = 'Business'
GROUP BY "departure_airport", "arrival_airport"
ORDER BY "business_revenue" DESC
LIMIT 10;
```

**Result (top 10 routes)**  

| from | to | business_revenue |
|------|----|------------------|
| DME  | KHV | 237,082,500 |
| KHV  | DME | 233,577,000 |
| KHV  | LED | 159,945,800 |
| LED  | KHV | 156,408,000 |
| LED  | IKT | 151,771,800 |
| IKT  | LED | 150,974,400 |
| DME  | OVB | 99,184,500 |
| OVB  | DME | 97,594,200 |
| IKT  | KZN | 89,167,800 |
| KZN  | IKT | 88,956,000 |

These are the routes generating the highest business‑class revenue in the dataset.

💭 _Reasoning: Need to explain? Just output result? Probably just show query and results._


In [42]:
#| hide
import nbdev; nbdev.nbdev_export()